## 카카오 API를 활용하여 충정로 역 주변 맛집 후기 검색

충정로역 주변 맛집 100개의 후기에서 별점, 내용, 가게이름 긁어오기.

카카오 api 사용.

충정로역 주변

1. 키워드 기반 로컬 검색 api
2. 키워드 설정
3. datas 에 json형태로 맛집 정보 저장
4. 맛집 고유 id, 를 datas 안에서 가져오기  
5. 맛집 id 내에서 코멘트가 더 이상 없을 때 까지(hasnext) 스크래이핑 

카카오 API 문서에 보면,
page라는 파라미터를 추가할 수 있어.

page=1 → 첫 번째 15개

page=2 → 다음 15개

page=3 → 그다음 15개 (※ 최대 3페이지까지만 지원해)

In [4]:
# 사전준비

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [19]:
# 나의 API키
REST_API_KEY = "668977541fc5a5fd5c0d271902df39c5"

# API 키를 헤더에 넣어서 보낼 준비
headers = {
    "Authorization": f"KakaoAK {REST_API_KEY}"
}

keywords = ["충정로역"]


KEYWORD_LOCAL_URL = "https://dapi.kakao.com/v2/local/search/keyword.json"


In [20]:


# 결과를 저장할 리스트
datas = []

# "충정로 맛집" 검색
search_word = keywords[0] + " 맛집"

for page in range(1, 4):  # 1, 2, 3 페이지 반복
    params = {
        "query": search_word,
        "radius": 1000,
        "page": page
    }
    
    response = requests.get(KEYWORD_LOCAL_URL, headers=headers, params=params)
    data = response.json().get('documents', [])
    
    datas.extend(data)  # 결과 합치기

print(datas)  # 최대 45개 가게 정보


[{'address_name': '서울 중구 만리동1가 53-8', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 일식 > 일본식라면', 'distance': '', 'id': '469577034', 'phone': '070-4177-0365', 'place_name': '유즈라멘 만리동점', 'place_url': 'http://place.map.kakao.com/469577034', 'road_address_name': '서울 중구 만리재로 217', 'x': '126.96823844833658', 'y': '37.556815391728634'}, {'address_name': '서울 중구 만리동1가 62-11', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 양식', 'distance': '', 'id': '1646052453', 'phone': '02-393-1824', 'place_name': '만리재비스트로', 'place_url': 'http://place.map.kakao.com/1646052453', 'road_address_name': '서울 중구 만리재로 201', 'x': '126.967831535859', 'y': '37.5556079449278'}, {'address_name': '서울 중구 만리동1가 62-27', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 한식', 'distance': '', 'id': '1541595421', 'phone': '010-6622-0195', 'place_name': '공일부엌', 'place_url': 'http://place.map.kakao.com/1541595421', 'road_add

In [21]:
print(len(datas))

45


In [22]:
COMMENT_URL = "https://place.map.kakao.com/m/commentlist/v/{}/{}?order=USEFUL&onlyPhotoComment=false"

In [23]:
# 1. 맛집 고유 id
id_place_list = [ { 'id' : data['id'], 'placename': data['place_name']} for data in datas ]
id_place_list

[{'id': '469577034', 'placename': '유즈라멘 만리동점'},
 {'id': '1646052453', 'placename': '만리재비스트로'},
 {'id': '1541595421', 'placename': '공일부엌'},
 {'id': '302317049', 'placename': '도마 중림점'},
 {'id': '221927024', 'placename': '국민회관'},
 {'id': '15982071', 'placename': '정원레스토랑 어반가든'},
 {'id': '8079539', 'placename': '중림장'},
 {'id': '2092235998', 'placename': '봉래축산'},
 {'id': '26939037', 'placename': '두툼'},
 {'id': '1981716686', 'placename': '서울부띠끄'},
 {'id': '1198183678', 'placename': '커피앤시가렛'},
 {'id': '25880887', 'placename': '철길떡볶이'},
 {'id': '412548224', 'placename': '풍문고기집'},
 {'id': '26644530', 'placename': '베리키친'},
 {'id': '15659337', 'placename': '미시령너머'},
 {'id': '11486129', 'placename': '가메골손왕만두 남대문본점'},
 {'id': '27276847', 'placename': '호수집 원조닭꼬치'},
 {'id': '820527185', 'placename': '이조식당'},
 {'id': '9991132', 'placename': '명인강메밀푸른초장'},
 {'id': '861738054', 'placename': '28총각'},
 {'id': '9923165', 'placename': '사조미가'},
 {'id': '1769574951', 'placename': '더하우스1932'},
 {'id': '23986974'

In [24]:
len(id_place_list)

45

In [56]:
import pandas as pd
import time

all_comment_df_list = []

# 전체 가게 수 저장
total_places = len(id_place_list[:45])

for idx, id_with_placename in enumerate(id_place_list[:45], 1):

    all_comment = []

    comment_id = 0
    has_next = True

    while has_next:
        try:
            SCRAP_COMMENT_URL = COMMENT_URL.format(id_with_placename['id'], comment_id)
            response = requests.get(SCRAP_COMMENT_URL, timeout=30)
            json_response = response.json()
        except Exception as e:
            print(f"⚠️ {id_with_placename['placename']} 요청 실패: {e}")
            break

        if 'comment' not in json_response:
            print(f"⚠️ {id_with_placename['placename']} 리뷰 없음, 스킵")
            break

        # ★ 이 두 줄 꼭 추가해야 해
        comment_datas = json_response['comment']
        comment_list = comment_datas['list']

        # 필요한 것만 골라서 저장
        for comment in comment_list:
            all_comment.append({
                '가게이름': id_with_placename['placename'],
                '리뷰내용': comment.get('contents', '').strip(),
                '리뷰별점': comment.get('point', None)
            })

        has_next = comment_datas.get('hasNext', False)
        
        if has_next and comment_list:
            comment_id = comment_list[-1]['commentid']
        else:
            has_next = False

    if all_comment:
        df = pd.DataFrame(all_comment)
        all_comment_df_list.append(df)
    
    time.sleep(1)
    print(f"가게 {total_places}개 중에, {idx}번째 완료! 1초 쉽니당 💤")

# 전체 리뷰 합치기
if all_comment_df_list:
    final_df = pd.concat(all_comment_df_list, ignore_index=True)
else:
    final_df = pd.DataFrame(columns=["가게이름", "리뷰내용", "리뷰별점"])

# 결과 확인
print(final_df.head())
print(f"리뷰 총 개수: {len(final_df)}개")


⚠️ 유즈라멘 만리동점 요청 실패: HTTPSConnectionPool(host='place.map.kakao.com', port=443): Max retries exceeded with url: /m/commentlist/v/469577034/12142939?order=USEFUL&onlyPhotoComment=false (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x123200090>, 'Connection to place.map.kakao.com timed out. (connect timeout=30)'))
가게 45개 중에, 1번째 완료! 1초 쉽니당 💤
가게 45개 중에, 2번째 완료! 1초 쉽니당 💤
가게 45개 중에, 3번째 완료! 1초 쉽니당 💤
가게 45개 중에, 4번째 완료! 1초 쉽니당 💤
가게 45개 중에, 5번째 완료! 1초 쉽니당 💤
가게 45개 중에, 6번째 완료! 1초 쉽니당 💤
⚠️ 중림장 리뷰 없음, 스킵
가게 45개 중에, 7번째 완료! 1초 쉽니당 💤
가게 45개 중에, 8번째 완료! 1초 쉽니당 💤
가게 45개 중에, 9번째 완료! 1초 쉽니당 💤
가게 45개 중에, 10번째 완료! 1초 쉽니당 💤
가게 45개 중에, 11번째 완료! 1초 쉽니당 💤
가게 45개 중에, 12번째 완료! 1초 쉽니당 💤
가게 45개 중에, 13번째 완료! 1초 쉽니당 💤
가게 45개 중에, 14번째 완료! 1초 쉽니당 💤
가게 45개 중에, 15번째 완료! 1초 쉽니당 💤
가게 45개 중에, 16번째 완료! 1초 쉽니당 💤
가게 45개 중에, 17번째 완료! 1초 쉽니당 💤
가게 45개 중에, 18번째 완료! 1초 쉽니당 💤
가게 45개 중에, 19번째 완료! 1초 쉽니당 💤
가게 45개 중에, 20번째 완료! 1초 쉽니당 💤
가게 45개 중에, 21번째 완료! 1초 쉽니당 💤
⚠️ 더하우스1932 리뷰 없음, 스킵
가게 45개 중에, 22번째 완료

In [57]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4830 entries, 0 to 4829
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   가게이름    4830 non-null   object
 1   리뷰내용    4830 non-null   object
 2   리뷰별점    4830 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 113.3+ KB


In [58]:

final_df.head()


,가게이름,리뷰내용,리뷰별점
0,유즈라멘 만리동점,평소 유자소스를 안좋아하는데 여기는 신세계..\n진짜 너무 맛있어요!! 캐치테이블있...,5
1,유즈라멘 만리동점,가장 좋아하는 전통적인 맛의 라멘집.,5
2,유즈라멘 만리동점,진짜 맛집이에요. 친절하고 분위기 좋고 단점은\n오래 웨이팅하는것 뿐입니다.,5
3,유즈라멘 만리동점,군대 제대 이후로 2열 종대로 줄서서 기다리다 식당 들어가기는 처음이다. 들어가면 ...,1
4,유즈라멘 만리동점,들어올 때 직원분들이 단체적으로 힘차게 인사해주셔서ㅋㅋ 되게 새롭고 힘받는 입장(E...,4


In [62]:
len(final_df["가게이름"].unique())

43

In [61]:
final_df

,가게이름,리뷰내용,리뷰별점
0,유즈라멘 만리동점,평소 유자소스를 안좋아하는데 여기는 신세계..\n진짜 너무 맛있어요!! 캐치테이블있...,5
1,유즈라멘 만리동점,가장 좋아하는 전통적인 맛의 라멘집.,5
2,유즈라멘 만리동점,진짜 맛집이에요. 친절하고 분위기 좋고 단점은\n오래 웨이팅하는것 뿐입니다.,5
3,유즈라멘 만리동점,군대 제대 이후로 2열 종대로 줄서서 기다리다 식당 들어가기는 처음이다. 들어가면 ...,1
4,유즈라멘 만리동점,들어올 때 직원분들이 단체적으로 힘차게 인사해주셔서ㅋㅋ 되게 새롭고 힘받는 입장(E...,4
...,...,...,...
4825,광화문국밥 본점,,4
4826,광화문국밥 본점,정갈한느낌 돼지국밥 \n돼지국밥이 살코기만 있으면서\n이렇게 정갈하고 시원할 수 있다니,5
4827,광화문국밥 본점,,4
4828,광화문국밥 본점,"깔끔한 국밥~\n가격9,000원저렴하나\n평범함~",3
